In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from My_Loss import HardTripletLoss2
from My_Loss import HardTripletLoss_D
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 200000
TEST_EPISODE = 1000
LEARNING_RATE =2e-5
Weight_Deacy = 1e-6
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'APY_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")
    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(64,1600,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1600,2048)
#triplet_network = TripletNetwork(attribute_network, metric_network)  #metric learning   
attribute_network.cuda(GPU) 
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=40000 , gamma=0.5)
metric_network_scheduler = StepLR(metric_network_optim , step_size=40000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024

        #print(batch_features_ext)

        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes,cos_sim = False):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)

        if cos_sim:
            relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        else:
            relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num = 0
#writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)
    

    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss_D(margin = Margin,cos_sim =True).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    metric_network.zero_grad()
    attribute_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()
    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        #writer.add_scalar('data/loss_zero_number', loss_zero_num, episode)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes,cos_sim =True)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(32), attributes,cos_sim =True)
        gzsl_seen_accuracy = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(32), attributes,cos_sim =True)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        #H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.0003, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.0706991299470122
gzsl: unseen=0.0173 , seen=0.0222 , h=0.0195
____________________________________________________________________________________________________
episode: 200 loss tensor(3.0001, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1780930584666943
gzsl: unseen=0.0620 , seen=0.4513 , h=0.1090
____________________________________________________________________________________________________
episode: 400 loss tensor(3.0001, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.22264395503645215
gzsl: unseen=0.0523 , seen=0.5406 , h=0.0953
____________________________________________________________________________________________________
episode: 600 loss tensor(3.0000, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 

episode: 5600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08615689600662402
gzsl: unseen=0.0344 , seen=0.0283 , h=0.0311
____________________________________________________________________________________________________
episode: 5800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08257817465851425
gzsl: unseen=0.0278 , seen=0.0423 , h=0.0335
____________________________________________________________________________________________________
episode: 6000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.07151259202542787
gzsl: unseen=0.0591 , seen=0.0265 , h=0.0365
____________________________________________________________________________________________________
episode: 6200 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 11400 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 11600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 11800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
_______________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 17000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 17200 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 17400 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
_______________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 22600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 22800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 23000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
_______________________

...

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 173800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 174000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 174200 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 179400 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 179600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 179800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 185000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 185200 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 185400 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 190600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 190800 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 191000 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________

class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 196200 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 196400 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________________________________________________________________________________________
episode: 196600 loss tensor(3., device='cuda:0')
loss_zero_number=  0
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.08333333333333333
gzsl: unseen=0.0000 , seen=0.0500 , h=0.0000
____________________